In [40]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import xarray as xr
import pathlib
import sys
import datetime as dt
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pickle

ROOT_PATH = pathlib.Path().resolve().parent
ROOT_PATH

sys.path.append(str(ROOT_PATH))

from modules.date_range import date_range


start = dt.date(2022,1,1)
end = dt.date(2023,1,1)
datasets = [xr.open_dataset(date.strftime(f"{ROOT_PATH}/data/%Y/%m/%Y_%m%d.nc"))[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf','ncld']].dropna(dim="time") for date in date_range(start,end)]
print("dataset Success")
del start,end


combined_dataset = xr.concat(datasets,dim="time")
print("combine Success")

del datasets


# 特徴量とターゲット変数に分割する
X = combined_dataset[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf']]
y = combined_dataset['ncld']

del combined_dataset


X_df = X.to_dataframe()
print("X_df Success")
del X


y_df = y.to_dataframe()
print("y_df Success")
del y


# データセットをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3)

del X_df,y_df
del X_train,y_train

In [2]:
### csvから
X_df = pd.read_csv(ROOT_PATH/"csv/2023-07-03X_df.csv")
y_df = pd.read_csv(ROOT_PATH/"csv/2023-07-03y_df.csv")["ncld"]

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.3,random_state=1)

In [42]:
X_test_drop = X_test.drop(["time","lat","lon"],axis=1)

In [31]:
X_test_drop["ncld"] = y_test
X_test.to_csv(ROOT_PATH/"csv/evalution_test.csv",index=False)

In [11]:
with open(ROOT_PATH/"models/model_using_msm_extract_only_amedas_point.pkl", 'rb') as f:
    model = pickle.load(f)

In [43]:
X_test_drop

,psea,sp,u,v,temp,rh,r1h,dswrf
357457,101316.972361,101247.706032,-1.944954,2.079511,279.483820,70.263762,-0.000007,266.6645
127869,102545.412705,101700.917032,-0.397554,1.204893,276.378993,77.527523,200.000000,665.0000
1358895,101456.880615,101385.320708,1.223242,-2.770642,296.468901,79.550459,200.000000,665.0000
579704,102533.944816,102351.375736,-0.525994,-3.333333,282.275029,42.660552,-0.000007,300.6945
567645,102951.834716,102712.843619,0.941896,-2.935780,278.122191,50.052753,200.000000,665.0000
...,...,...,...,...,...,...,...,...
131374,102553.210870,100992.660165,2.360856,-1.394495,279.376667,65.910551,-0.000007,0.0005
1571181,102076.605375,101828.439966,-0.342508,1.223242,289.388953,72.493119,200.000000,665.0000
1040615,101763.761344,101088.990438,-0.801223,2.336392,298.985693,91.259174,-0.000007,318.9805
1609801,101786.238407,101589.907860,-2.201835,-1.810398,297.577021,76.522936,-0.000007,605.6730


In [44]:
y_test

357457     55.126666
127869     28.320001
1358895    61.034999
579704      2.001669
567645      9.716668
             ...    
131374      0.928336
1571181     3.856669
1040615    38.818334
1609801     1.855002
1037549    20.751668
Name: ncld, Length: 596556, dtype: float64

In [45]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

y_pred = model.predict(X_test_drop)

In [46]:
y_test

357457     55.126666
127869     28.320001
1358895    61.034999
579704      2.001669
567645      9.716668
             ...    
131374      0.928336
1571181     3.856669
1040615    38.818334
1609801     1.855002
1037549    20.751668
Name: ncld, Length: 596556, dtype: float64

In [16]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test.values,y_pred))
r2 = r2_score(y_test.values,y_pred)

In [37]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 23.543900686948984
R2: 0.45154444286971285


In [17]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 21.228720399427075
R2: 0.5636176913510266


In [47]:
ans = X_test.copy()
ans["predict_ncld"] = y_pred
ans["correct_ncld"] = y_test
ans["ncld_diff"] = y_test - y_pred

In [49]:
ans.to_csv(ROOT_PATH/"csv/ans_use_msm_extract_anedas_point.csv",index=False)